In [31]:
# from google.colab import drive
# drive.mount('/content/drive')

In [32]:
# !pip install ultralytics -q

In [33]:
from ultralytics import YOLO
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import math

In [34]:
image_path = [
    "/content/drive/MyDrive/Projects/japan/bolt/Image-2024-03-27-13-42-44-png-rf-91f7dde07ae4db4bb3a222f73c448e21.jpg",
    "/content/drive/MyDrive/Projects/japan/bolt/Image__2024-03-27__13-42-52.png",
    "/content/drive/MyDrive/Projects/japan/bolt/Image__2024-03-27__13-42-27.png",
    "/content/drive/MyDrive/Projects/japan/bolt/Image__2024-03-27__13-43-22.png",
    ]

In [35]:
pose_model = YOLO("/content/drive/MyDrive/Projects/japan/pose/best.pt")
obb_model = YOLO("/content/drive/MyDrive/Projects/japan/obb/best1.pt")

model = [obb_model,pose_model]

In [36]:
def line_angle(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.atan2(dy, dx)

def CenterAndAngle(img,model):
  dec = []
  obb_predict = model[0].predict(img)
  pose_predict = model[1].predict(img)
  obb_points = obb_predict[0].obb.xyxyxyxy.cpu().numpy().astype(int)
  pose_points = pose_predict[0].keypoints.xy.cpu().numpy().astype(int)


  for obb_point in obb_points:

    # Find Top Mid point
    xmidtop = int((obb_point[0][0] + obb_point[1][0]) / 2)
    ymidtop = int((obb_point[0][1] + obb_point[1][1]) / 2)

    # Find Bottom Mid point
    xmidbot = int((obb_point[2][0] + obb_point[3][0]) / 2)
    ymidbot = int((obb_point[2][1] + obb_point[3][1]) / 2)

    # Find Center point
    xmidmid = int((xmidtop + xmidbot) / 2)
    ymidmid = int((ymidtop + ymidbot) / 2)

    closest_point = {}
    for pose_point in pose_points:
      X = int (abs(pose_point[0][0]-xmidmid))
      Y = int (abs(pose_point[0][1]-ymidmid))
      D = X+Y
      closest_point.update({D:pose_point})

    min_D = min(closest_point.keys())
    pose_point = closest_point[min_D]

    # Calculate ManHattan Distance From POSE Head
    p1 = int (abs(pose_point[0][0]-obb_point[0][0]))
    q1 = int (abs(pose_point[0][1]-obb_point[0][1]))
    p2 = int (abs(pose_point[0][0]-obb_point[3][0]))
    q2 = int (abs(pose_point[0][1]-obb_point[3][1]))
    d1 = p1+q1
    d2 = p2+q2

    if d1<d2:
      cx = xmidtop
      cy = ymidtop

    else:
      cx = xmidbot
      cy = ymidbot

    angle_redious = line_angle(xmidmid, ymidmid, cx, cy)

    angle = -1*(math.degrees(angle_redious))

    if angle<0:
      angle = 360+angle

    dec.append({"Center": {"X": xmidmid,"Y": ymidmid},"Angle": "{:.2f}".format(angle)})

  return dec


for img in image_path:
 print(CenterAndAngle(img,model))


image 1/1 /content/drive/MyDrive/Projects/japan/bolt/Image-2024-03-27-13-42-44-png-rf-91f7dde07ae4db4bb3a222f73c448e21.jpg: 640x640 255.8ms
Speed: 3.2ms preprocess, 255.8ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/Projects/japan/bolt/Image-2024-03-27-13-42-44-png-rf-91f7dde07ae4db4bb3a222f73c448e21.jpg: 640x640 3 Bolts, 214.8ms
Speed: 3.0ms preprocess, 214.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
[{'Center': {'X': 215, 'Y': 137}, 'Angle': '46.10'}, {'Center': {'X': 153, 'Y': 452}, 'Angle': '203.96'}, {'Center': {'X': 385, 'Y': 314}, 'Angle': '125.36'}]

image 1/1 /content/drive/MyDrive/Projects/japan/bolt/Image__2024-03-27__13-42-52.png: 480x640 174.2ms
Speed: 3.9ms preprocess, 174.2ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/MyDrive/Projects/japan/bolt/Image__2024-03-27__13-42-52.png: 480x640 1 Bolt, 212.2ms
Speed: 4.2ms preprocess, 212.2ms inference